# Retrieval Augmented Generation

## Setup API clients

In [4]:
import os

import azure.identity
import dotenv
from azure.search.documents import SearchClient
from azure.search.documents.models import VectorizedQuery
from openai import OpenAI

dotenv.load_dotenv()

azure_credential = azure.identity.AzureDeveloperCliCredential(tenant_id=os.getenv("AZURE_TENANT_ID"))

# Initialize Azure OpenAI client
AZURE_OPENAI_SERVICE = os.getenv("AZURE_OPENAI_SERVICE")
AZURE_OPENAI_EMBEDDING_DEPLOYMENT = os.getenv("AZURE_OPENAI_EMBEDDING_DEPLOYMENT")

token_provider = azure.identity.get_bearer_token_provider(azure_credential, "https://cognitiveservices.azure.com/.default")
openai_client = OpenAI(
    base_url=f"https://{AZURE_OPENAI_SERVICE}.openai.azure.com/openai/v1",
    api_key=token_provider)

def get_embedding(text):
    get_embeddings_response = openai_client.embeddings.create(model=AZURE_OPENAI_EMBEDDING_DEPLOYMENT, input=text)
    return get_embeddings_response.data[0].embedding

# Initialize Azure search client
AZURE_SEARCH_SERVICE = os.getenv("AZURE_SEARCH_SERVICE")
AZURE_SEARCH_ENDPOINT = f"https://{AZURE_SEARCH_SERVICE}.search.windows.net"
AZURE_SEARCH_FULL_INDEX = os.getenv("AZURE_SEARCH_INDEX", "gptkbindex")
AZURE_SEARCH_FIELD_NAME_EMBEDDING = os.getenv("AZURE_SEARCH_FIELD_NAME_EMBEDDING", "embedding")

search_client = SearchClient(AZURE_SEARCH_ENDPOINT, AZURE_SEARCH_FULL_INDEX, credential=azure_credential)

## Prepare user question

In [5]:
user_question = "What does a product manager do?"
user_question_vector = get_embedding(user_question)

## Retrieve matching documents

The search call below does a **hybrid search**, performing both a full-text search and a vector search in parallel.
It merges those results using Reciprocal Rank Fusion (RRF). 
Finally, it re-ranks the merged results using the AI Search semantic ranker, a re-ranking model.

In [6]:
r = search_client.search(
        user_question,
        top=5, 
        vector_queries=[
                VectorizedQuery(vector=user_question_vector, k_nearest_neighbors=50, fields=AZURE_SEARCH_FIELD_NAME_EMBEDDING)],
        query_type="semantic",
        semantic_configuration_name="default")

sources = "\n\n".join([f"[{doc['sourcepage']}]: {doc['content']}\n" for doc in r])

print(sources)

[role_library.pdf#page=29]: · Knowledge of industry trends, technologies, and regulations.

· Excellent communication and presentation skills.


#### Manager of Product Management

Job Title: Manager of Product Management, Contoso Electronics

Job Summary:

The Manager of Product Management is responsible for overseeing the product
management team, driving product development and marketing strategy for Contoso
Electronics. This individual will be accountable for the successful launch of new products
and the implementation of product life-cycle management processes. The Manager of
Product Management will collaborate with internal teams, such as engineering, sales,
marketing, and finance, as well as external partners, suppliers, and customers to ensure
successful product execution.

Responsibilities:

· Lead the product management team and provide guidance on product strategy, design,
development, and launch.

· Develop and implement product life-cycle management processes.

· Monitor an

## Send question and documents to the model

In [7]:
SYSTEM_MESSAGE = """
Assistant helps company employees questions about the employee handbook. Be brief in your answers.
Answer ONLY with the facts listed in the list of sources below.
If there isn't enough information below, say you don't know. Do not generate answers that don't use the sources below.
Each source has a name followed by colon and the actual information, include the source name for each fact you use.
Use square brackets to reference the source, for example [info1.txt].
"""
USER_MESSAGE = user_question + "\nSources: " + sources

# Now we can use the matches to generate a response
response = openai_client.chat.completions.create(
    model=os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME"),
    temperature=0.7,
    messages=[
        {"role": "system", "content": SYSTEM_MESSAGE},
        {"role": "user", "content": USER_MESSAGE},
    ],
)

answer = response.choices[0].message.content
print(answer)

A product manager at Contoso Electronics is responsible for developing and implementing product strategies and plans that support business objectives and growth. They lead product management teams, manage product life cycles from concept to launch, and oversee product roadmaps, requirements, and portfolios. They research market trends and customer needs to identify new product opportunities, develop pricing and promotional strategies, and collaborate with internal teams for successful product execution. They also monitor product performance and customer feedback to make improvements, manage budgets, and analyze competitive landscapes to keep products competitive [role_library.pdf#page=17, #page=23, #page=29].
